In [ ]:
import json
import pandas as pd

# Cargar el archivo Excel con los códigos ATC
atc_df = pd.read_excel("../../assets/Tabla_ATC.xlsx")

# Crear los diccionarios a partir del Excel
# Ajustamos los nombres de las columnas según la estructura proporcionada
grupo_anatomico_dict = {}  # Nivel (anatómico)
subgrupo_terapeutico_L2_dict = {}  # Nivel 2 (Subgrupo terapéutico)
subgrupo_terapeutico_L3_dict = {}  # Nivel 3 (Subgrupo terapéutico o farmacológico)
subgrupo_terapeutico_L4_dict = (
    {}
)  # Nivel 4 (Subgrupo terapéutico, farmacológico o químico)

# Mapear grupos anatómicos (nivel 1)
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[0], atc_df.columns[2]]
).iterrows():
    if pd.notna(row[atc_df.columns[0]]) and pd.notna(row[atc_df.columns[2]]):
        grupo_anatomico_dict[row[atc_df.columns[0]]] = row[atc_df.columns[2]]

# Mapear subgrupos terapéuticos nivel 2
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[3], atc_df.columns[5]]
).iterrows():
    if pd.notna(row[atc_df.columns[3]]) and pd.notna(row[atc_df.columns[5]]):
        subgrupo_terapeutico_L2_dict[row[atc_df.columns[3]]] = row[atc_df.columns[5]]

# Mapear subgrupos terapéuticos nivel 3
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[6], atc_df.columns[8]]
).iterrows():
    if pd.notna(row[atc_df.columns[6]]) and pd.notna(row[atc_df.columns[8]]):
        subgrupo_terapeutico_L3_dict[row[atc_df.columns[6]]] = row[atc_df.columns[8]]

# Mapear subgrupos terapéuticos nivel 4
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[9], atc_df.columns[11]]
).iterrows():
    if pd.notna(row[atc_df.columns[9]]) and pd.notna(row[atc_df.columns[11]]):
        subgrupo_terapeutico_L4_dict[row[atc_df.columns[9]]] = row[atc_df.columns[11]]


# Función para mapear el código ATC
def mapear_atc(ATC):
    if ATC and isinstance(ATC, str) and ATC != "Desconocido":
        L1 = ATC[0] if len(ATC) >= 1 else None  # Nivel (anatómico)
        L2 = ATC[:3] if len(ATC) >= 3 else None  # Nivel 2 (Subgrupo terapéutico)
        L3 = (
            ATC[:4] if len(ATC) >= 4 else None
        )  # Nivel 3 (Subgrupo terapéutico o farmacológico)
        L4 = (
            ATC[:5] if len(ATC) >= 5 else None
        )  # Nivel 4 (Subgrupo terapéutico, farmacológico o químico)

        grupo_anat = grupo_anatomico_dict.get(L1, "Desconocido")
        subgr_ter_L2 = subgrupo_terapeutico_L2_dict.get(L2, "Desconocido")
        subgr_ter_L3 = subgrupo_terapeutico_L3_dict.get(L3, "Desconocido")
        subgr_ter_L4 = subgrupo_terapeutico_L4_dict.get(L4, "Desconocido")

        return L1, grupo_anat, L2, subgr_ter_L2, L3, subgr_ter_L3, L4, subgr_ter_L4
    else:
        return (
            None,
            "Desconocido",
            None,
            "Desconocido",
            None,
            "Desconocido",
            None,
            "Desconocido",
        )


# Cargar el archivo JSON con las fichas técnicas
with open(
    "../../data/outputs/2_data_preprocessing/wrangler/medicamentos.json",
    "r",
    encoding="utf-8",
) as f:
    datos_json = json.load(f)

medicamentos_list = []
for nombre_archivo, datos in datos_json.items():
    datos["nombre_archivo"] = nombre_archivo
    medicamentos_list.append(datos)

medicamentos_df = pd.DataFrame(medicamentos_list)
medicamentos_df = medicamentos_df.drop_duplicates()

if medicamentos_df["ATC"].isnull().any():
    print("Hay valores nulos en 'ATC', se van a llenar con 'Desconocido'.")

medicamentos_df = medicamentos_df.fillna({"ATC": "Desconocido"})

mapeo_atc = medicamentos_df["ATC"].apply(mapear_atc)

medicamentos_df[
    [
        "ATC_Nivel_Anatomico",
        "Descripcion_Nivel_Anatomico",
        "ATC_Nivel_2_Subgrupo_Terapeutico",
        "Descripcion_Nivel_2_Subgrupo_Terapeutico",
        "ATC_Nivel_3_Subgrupo_Terapeutico_Farmacologico",
        "Descripcion_Nivel_3_Subgrupo_Terapeutico_Farmacologico",
        "ATC_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico",
        "Descripcion_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico",
    ]
] = pd.DataFrame(mapeo_atc.tolist(), index=medicamentos_df.index)

print("\nPrimeras filas del DataFrame final con mapeo de ATC:")
display(medicamentos_df.head())

medicamentos_df.to_json(
    "../../data/outputs/2_data_preprocessing/fichas_tecnicas_mapped_atc.json",
    orient="records",
    lines=False,
    force_ascii=False,
    indent=4,
)

print("\nDataFrame guardado correctamente en el archivo JSON.")


Primeras filas del DataFrame final con mapeo de ATC:


,indicaciones,posologia,contraindicaciones,advertencias,interacciones,fertilidad_embarazo,efectos_conducir,reacciones_adversas,sobredosis,ATC,...,fecha_revision,nombre_archivo,ATC_Nivel_Anatomico,Descripcion_Nivel_Anatomico,ATC_Nivel_2_Subgrupo_Terapeutico,Descripcion_Nivel_2_Subgrupo_Terapeutico,ATC_Nivel_3_Subgrupo_Terapeutico_Farmacologico,Descripcion_Nivel_3_Subgrupo_Terapeutico_Farmacologico,ATC_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico,Descripcion_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico
0,ácido alendrónico/colecalciferol est á indicad...,posología la dosis recomendada es un comprimid...,hipersensibilidad a los principios activos o a...,alendronato reacciones adversas de la parte su...,"alendronato es probable que, si se toman al mi...",ácido alendr ónico/colecalciferol sólo est á i...,la influencia de ácido alendr ónico/colecalcif...,resumen del perfil de seguridad las reacciones...,alendronato síntomas como consecuencia de la s...,A02BC03,...,None,ACIDO_ALENDRONICO_COLECALCIFEROL_SEMANAL_HEMON...,A,ALIMENTARY TRACT AND METABOLISM,A02,DRUGS FOR ACID RELATED DISORDERS,A02B,DRUGS FOR PEPTIC ULCER AND GASTRO-OESOPHAGEAL ...,A02BC,Proton pump inhibitors
1,actynox está indicado para el tratamiento a co...,poso logía la duración de la administración de...,"cuando se inhala actynox, las burbujas de gas ...",actynox debe ser administrado solamente por pe...,combinación con otros medicamentos el componen...,embarazo existe un elevado número de datos en ...,el componente óxido nitroso de actynox tiene e...,se ha notificado anemia megaloblástica y leuco...,como se requiere la participación del paciente...,Desconocido,...,None,ACTYNOX_50%_50%_GAS_COMPRIMIDO_MEDICINAL.txt,None,Desconocido,None,Desconocido,None,Desconocido,None,Desconocido
2,tratamiento de estados graves de deficiencia d...,vía intravenosa. posología adultos y adolescen...,- hipersensibilidad al ácido ascórbico o a alg...,- el ácido ascórbico no debe administrarse en ...,la acidificación de la orina que produce la ad...,embarazo no hay estudios controlados relativos...,el producto tiene un efecto nulo o insignifica...,las reacciones adversas enumeradas han sido id...,no hay evidencia que este producto pueda produ...,Desconocido,...,None,ACIDO_ASCORBICO_BAYER_1000_mg_5_ml_SOLUCION_IN...,None,Desconocido,None,Desconocido,None,Desconocido,None,Desconocido
3,"tratamiento, en pacientes inmunocompetentes, d...",posología adultos aplicar cinco veces al día a...,None,no se recomienda la aplicación cutánea de acic...,"la absorción por vía cutánea es mínima, por lo...",embarazo no se ha establecido la seguridad del...,la influencia de este medicamento sobre la cap...,se ha utilizado la siguiente terminología para...,si se ingirieran por vía oral l0 g de este med...,Desconocido,...,None,ACICLOVIR_COMBIX_50_mg_g_CREMA_EFG.txt,None,Desconocido,None,Desconocido,None,Desconocido,None,Desconocido
4,tratamiento de la epilepsia parcial o generali...,posología ácido valproico debe ser administrad...,ácido valproico está contraindicado en las sig...,daño hepático grave condiciones de aparición e...,efecto de valproato sobre otros medicamentos a...,embarazo valproato está contraindicado como tr...,se debe advertir a los pacientes sobre el ries...,se utiliza la convención de frecuencia meddra:...,los signos de intoxicación masiva aguda normal...,Desconocido,...,None,ACIDO_VALPROICO_AUROVITAS_500_MG_COMPRIMIDOS_D...,None,Desconocido,None,Desconocido,None,Desconocido,None,Desconocido



DataFrame guardado correctamente en el archivo JSON.
